<a href="https://colab.research.google.com/github/shirounk01/Text2SQL/blob/main/QuestionAnsweringTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install word2number

#Roberta-Base-Squad2
Score: 74.15349887

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from word2number import w2n
import re

def findIndex(arr, warr):
    index = []
    for i in range(len(arr)):
        for j in range(len(warr)):
            if arr[i]==warr[j]:
                index.append(i)
    index.sort()
    return index

signs = {
    '': '',
    '=': '',
    '!=': 'not ',
    '>': 'more than ',
    '<': 'less than ',
    '>=': 'more or equal than ',
    '<=': 'less or equal than ',
    'LIKE': 'like ',
    'like': 'like '
}

rules = {                                               # setul de reguli: daca e cuvantul x, se pune intrabrea respectiva
    'WHERE': '{} is {}what?',
    'AND': '{} is {}what?',
    'OR': '{} is {}what?',
    'HAVING': '{} is {}what?',
    'Count(*)': '{} is {}what?',
    'LIKE': '{} is {}like what?',
    'BETWEEN': '{} is between {}what?',
    'where': '{} is {}what?',
    'and': '{} is {}what?',
    'or': '{} is {}what?',
    'having': '{} is {}what?',
    'count(*)': '{} is {}what?',
    'like': '{} is {}like what?',
    'between': '{} is between {}what?'
}

def question(word: str, subject: str, quantity = ''):
    try:
        return rules[word].format(subject, signs[quantity])
    except:
        return ' '

def answer(question: str, context: str):
    model_name = "deepset/roberta-base-squad2"
    
    nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
    QA_input = {
        'question': question,
        'context': context
    }
    res = nlp(QA_input)
    
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return res["answer"]

from google.colab import auth
import re
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('DATASET GOLD_3').sheet1
r = 28
rows = worksheet.get_all_values()
while r <= 443:
    init = rows[r][1]
    context = rows[r][0]
    sql = ['SELECT', 'FROM', 'HAVING', 'JOIN', 'JOIN LEFT', 'JOIN RIGHT', 'ON', 'AND', 'OR', 'WHERE',  'INTERSECT', 'Count(*)', 'BETWEEN', \
           'select', 'from', 'having', 'join', 'join left', 'join right', 'on', 'and', 'or', 'where', 'intersect', 'count(*)', 'between']
    exclude = [',',':',';','?']
    query=init                                              # copiaza textul de query
    for x in exclude:                                       # modifica toate chestiile de nu ne trebuie cu spatiu
        query = query.replace(x, ' ')
    query=query.split()                                     # se sparge in list
    print(query)
    sqlIndex = findIndex(query, sql)                        # se cauta pozitiile cuvintelor cheie
    terminalIndex = findIndex(query, ["'terminal'"])      # se cauta pozitiile cuvantului terminal
    pairIndex = []
    for x in terminalIndex:                                 # se fac perechi de forma: ultimul cuvant cheie dinaintea lui terminal, terminal
        last = 0
        for y in sqlIndex:
            if y > x:
                break
            last = y
        pairIndex.append((last, x))
    replace = []
    for pair in pairIndex:                                  # pentru fiecare pereche gasita, se face intrebarea, se raspunde si se ia raspunsul
        q = query[pair[0]]
        s = ' '.join(query[pair[0]+1:pair[1]])
        quant = query[pair[1]-1]
        if quant == q:
            quant = ''
        ans = answer(question(q, s, quant), context)

        copy = re.split(' and | or | to ', ans)
        for i in range(len(copy)):
            try:
                copy[i] = w2n.word_to_num(copy[i])
            except:
                copy[i] = ''.join(re.findall(r'\d+', copy[i]))
        try:
            copy = list(filter(('').__ne__, copy))
        except:
            pass
        if len(copy) > 0:
            for j in range(len(copy)):
                copy[j] = str(copy[j])
            replace+=copy
        else:
            copy = re.split(' and | or ', ans)
            replace += copy

    for x in replace:                                       # pentru fiecare raspuns, se modifica cuvintele terminal pe rand
        init = init.replace("'terminal'","'{}'".format(x) if x.isnumeric()==False else x, 1)
    print(init)
    cell = worksheet.range('D{}:D{}'.format(r+1, r+1))
    cell[0].value = init
    worksheet.update_cells(cell)
    r+=1


#GPT_NEO
Score: 45.3724605

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import re

def findIndex(arr, warr):
    index = []
    for i in range(len(arr)):
        for j in range(len(warr)):
            if arr[i]==warr[j]:
                index.append(i)
    index.sort()
    return index

signs = {
    '': '',
    '=': '',
    '!=': 'not ',
    '>': 'more than ',
    '<': 'less than ',
    '>=': 'more or equal than ',
    '<=': 'less or equal than ',
    'LIKE': 'like ',
    'like': 'like '
}

rules = {                                               # setul de reguli: daca e cuvantul x, se pune intrabrea respectiva
    'WHERE': '{} is {}what?',
    'AND': '{} is {}what?',
    'OR': '{} is {}what?',
    'HAVING': '{} is {}what?',
    'Count(*)': '{} is {}what?',
    'LIKE': '{} is {}like what?',
    'BETWEEN': '{} {}',
    'where': '{} is {}what?',
    'and': '{} is {}what?',
    'or': '{} is {}what?',
    'having': '{} is {}what?',
    'count(*)': '{} is {}what?',
    'like': '{} is {}like what?',
    'between': '{} {}'
}

def question(word: str, subject: str, quantity = ''):
    try:
        return rules[word].format(subject, signs[quantity])
    except:
        return ''

def answer(question: str, context: str):
    train = ""
    with open("/content/drive/MyDrive/terminal/test.txt") as file:
        train = file.read()
    prompt = train+'\n'+context+' => '+question+' => '
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=input_ids.shape[1]+20)
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    print(gen_text)
    string = gen_text[len(prompt):len(prompt)+gen_text[len(prompt):].find('\n#')]
    if len(string):
        if string[0]==' ':
            string = string[1:]
    return string

from google.colab import auth
import re
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('DATASET GOLD').sheet1
r = 419
rows = worksheet.get_all_values()

questions = []
while r <= 443:
    init = rows[r][1]
    context = rows[r][0]
    sql = ['SELECT', 'FROM', 'HAVING', 'JOIN', 'JOIN LEFT', 'JOIN RIGHT', 'ON', 'AND', 'OR', 'WHERE', 'ASC', 'DESC', 'INTERSECT', 'Count(*)', 'BETWEEN', \
           'select', 'from', 'having', 'join', 'join left', 'join right', 'on', 'and', 'or', 'where', 'asc', 'desc', 'intersect', 'count(*)', 'between']
    exclude = [',',':',';','?']
    query=init                                              # copiaza textul de query
    for x in exclude:                                       # modifica toate chestiile de nu ne trebuie cu spatiu
        query = query.replace(x, ' ')
    query=query.split()                                     # se sparge in list
    #print(query)
    sqlIndex = findIndex(query, sql)                        # se cauta pozitiile cuvintelor cheie
    terminalIndex = findIndex(query, ["'terminal'"])      # se cauta pozitiile cuvantului terminal
    pairIndex = []
    for x in terminalIndex:                                 # se fac perechi de forma: ultimul cuvant cheie dinaintea lui terminal, terminal
        last = 0
        for y in sqlIndex:
            if y > x:
                break
            last = y
        pairIndex.append((last, x))
    replace =[]
    for pair in pairIndex:                                  # pentru fiecare pereche gasita, se face intrebarea, se raspunde si se ia raspunsul
        q = query[pair[0]]
        s = ' '.join(query[pair[0]+1:pair[1]-1])
        quant = query[pair[1]-1]
        if quant == q:
            quant = ''
        qq = question(q, s, quant)
        if qq == '':
            continue
        print(qq)
        ans = answer(qq, context)
        replace.append(ans)
    print(query)
    print(replace)
    for x in replace:                                       # pentru fiecare raspuns, se modifica cuvintele terminal pe rand
        init = init.replace("'terminal'","'{}'".format(x) if x.isnumeric()==False else x, 1)
    print(init)
    cell = worksheet.range('D{}:D{}'.format(r+1, r+1))
    cell[0].value = init
    worksheet.update_cells(cell)    
    r+=1
    